# agentscope.pipeline

agentscope.pipeline 模块提供了用于构建 多智能体协作流程 的高级工具，包括消息广播、顺序执行、并行分发、流式输出等功能。它通过上下文管理器（如 MsgHub）和异步管道（如 SequentialPipeline、FanoutPipeline）简化复杂多智能体系统的编排。

## 1. MsgHub：自动消息广播中心 
在 with MsgHub(...): 块中，任意智能体的回复会自动广播给所有参与者。等价于手动调用 observe()，但更简洁。

参数 
- participants: 参与的智能体列表（List[AgentBase]）
- announcement: 进入上下文时广播的初始消息（可选）
- enable_auto_broadcast: 是否启用自动广播（默认 True）
- name: 名称（用于调试）
     

In [6]:
from agentscope.agent import ReActAgent
from agentscope.message import Msg
from agentscope.pipeline import MsgHub
from agentscope.model import DashScopeChatModel
from agentscope.formatter import DashScopeChatFormatter
import os
from dotenv import load_dotenv
load_dotenv()
# 初始化模型（需配置 API key）
model = DashScopeChatModel(
            model_name="qwen-max",
            api_key=os.environ["DASHSCOPE_API_KEY"],
            stream=False,
        )
formatter = DashScopeChatFormatter()

# 创建两个智能体
alice = ReActAgent(name="Alice", sys_prompt="你叫 Alice，性格开朗。", model=model, formatter=formatter)
bob = ReActAgent(name="Bob", sys_prompt="你叫 Bob，喜欢提问。", model=model, formatter=formatter)

# 使用 MsgHub 自动广播（不使用 with 语句）
async def multi_agent_chat():
    # 创建 MsgHub 实例
    hub = MsgHub(participants=[alice, bob])
    
    # Alice 先发言
    msg1 = await alice.reply(Msg("user", "大家好！", "user"))
    print(f"Alice 说: {msg1.content}")
    
    # 手动让 Bob 观察 Alice 的消息
    bob.observe(msg1)
    
    # Bob 基于上下文回复
    msg2 = await bob.reply()
    print(f"Bob 说: {msg2.content}")

# 在 Jupyter 中直接使用 await
await multi_agent_chat()

Alice: 你好！很高兴见到你。有什么我可以帮到你的吗？
Alice 说: 你好！很高兴见到你。有什么我可以帮到你的吗？


/tmp/ipykernel_167830/744756880.py:31: RuntimeWarning: coroutine 'ReActAgent.observe' was never awaited
  bob.observe(msg1)


Bob: 您好！有什么问题我可以帮到您吗？
Bob 说: 您好！有什么问题我可以帮到您吗？


二、SequentialPipeline：顺序流水线 
功能说明 

    将多个智能体串行执行，前一个的输出作为后一个的输入。
    最终返回最后一个智能体的输出。
     

函数式 vs 类式 

    sequential_pipeline(agents, msg)：一次性函数调用（推荐简单场景）
    SequentialPipeline 类：可复用、配置化（适合复杂系统）
     

In [7]:
from agentscope.pipeline import sequential_pipeline

# 假设已创建 agent1, agent2, agent3（类型为 ReActAgent 或 UserAgent）
msg_input = Msg("user", "请总结以下内容：AgentScope 是一个智能体框架。", "user")

msg_output = await sequential_pipeline(
    agents=[alice,bob],
    msg=msg_input
)
print("最终结果:", msg_output.content)

Alice: AgentScope 是一个用于创建智能体的框架。它提供了一套工具和结构，帮助开发者设计和实现具备特定功能的智能体系统。简而言之，它简化了构建智能体软件的过程。
如果需要更详细的总结或有其他具体方面想要了解，请告诉我！
Bob: 您是想要了解 AgentScope 的具体应用案例还是其他相关信息呢？
最终结果: 您是想要了解 AgentScope 的具体应用案例还是其他相关信息呢？


三、FanoutPipeline：并行分发（扇出） 
功能说明 

    将同一输入分发给多个智能体。
    支持 并发执行（默认）或 顺序执行。
    返回所有智能体的输出列表。
     

两种使用方式 

    函数式：fanout_pipeline(agents, msg, enable_gather=True)
    类式：FanoutPipeline(agents, enable_gather=True)，可复用
     

In [9]:
from agentscope.pipeline import fanout_pipeline

# 创建三个“专家”智能体
expert1 = ReActAgent(
    name="MathExpert",
    sys_prompt="你是数学专家，擅长解答数学问题。",
    model=model,
    formatter=formatter
)

expert2 = ReActAgent(
    name="HistoryExpert",
    sys_prompt="你是历史专家，精通世界历史和中国历史。",
    model=model,
    formatter=formatter
)

expert3 = ReActAgent(
    name="ScienceExpert",
    sys_prompt="你是科学专家，熟悉物理、化学和生物知识。",
    model=model,
    formatter=formatter
)

question = Msg("user", "请从你的专业角度解释‘人工智能’。", "user")

# 并发执行（默认）
responses = await fanout_pipeline(
    agents=[expert1, expert2, expert3],
    msg=question
)

for i, resp in enumerate(responses):
    print(f"专家 {i+1} 回答: {resp.content}")

ScienceExpert: 人工智能（Artificial Intelligence，简称AI）是一个跨学科的领域，它结合了计算机科学、数学、统计学、逻辑学、哲学、心理学等多方面的知识。从科学专家的角度来看，人工智能可以被定义为一种使机器能够执行通常需要人类智能才能完成的任务的技术。

在物理层面，人工智能依赖于复杂的计算硬件，这些硬件能够快速处理大量数据和执行复杂算法。这包括中央处理器（CPU）、图形处理器（GPU），以及专门设计用于加速机器学习任务的张量处理单元（TPU）等。

在化学层面，虽然直接与人工智能相关的化学内容较少，但是半导体材料的研究与发展对于制造高性能的电子设备至关重要，而这些设备是支撑现代人工智能技术的基础设施。此外，随着纳米技术和新材料的发展，未来可能会有更加高效和小型化的计算元件出现，从而推动人工智能的进步。

生物角度的人工智能则涉及到神经网络的设计，这是一种受到人脑结构启发的计算模型。人工神经网络模仿了大脑中神经元之间的连接方式，通过层次化的方式组织起来，以便于处理信息和学习数据中的模式。深度学习就是一种基于深层神经网络的方法，它已经在图像识别、语音识别、自然语言处理等领域取得了显著成就。

总的来说，人工智能是一种模拟人类智能行为的技术，它通过算法和大量的数据训练来实现诸如感知环境、理解语言、解决问题、学习新知等功能。随着技术的发展，人工智能正在变得越来越强大，并且在医疗、教育、交通、娱乐等多个行业中发挥着重要作用。同时，这也带来了关于隐私、伦理和社会影响等方面的讨论。
HistoryExpert: 作为一位历史专家，我的专业领域主要集中在对过去事件、文化、社会结构以及人类行为的研究上。然而，“人工智能”（Artificial Intelligence, AI）是一个现代科技概念，属于计算机科学和工程学的范畴。尽管如此，我可以尝试从一个跨学科的角度来提供一些关于人工智能的基本解释。

人工智能是指由人制造出来的具有一定智能的系统，它能够理解复杂的环境信息，并且可以像人一样进行思考、学习和解决问题。AI系统通常包括以下几个关键方面：

1. **机器学习**：这是一种让计算机通过数据而不是明确编程指令来学习的技术。机器学习算法可以从大量数据中提取模式，然后使用这些模式来进行预测或决策。
2. **深度学习**：这是机器学习的一个子集，